In [1]:
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting
from keras.models import load_model
import cv2

Using TensorFlow backend.


In [2]:
lookup = {'Fist': 0, 'Nothing': 1, 'Palm': 2, 'Swing': 3, 'Thumb': 4, 'Yo': 5}

In [3]:
reverselookup = {0: 'Fist', 1: 'Nothing', 2: 'Palm', 3: 'Swing', 4: 'Thumb', 5: 'Yo'}

In [4]:
#To load saved data
x_train = np.load("X_TRAIN.npy")
y_train = np.load("Y_TRAIN.npy")
x_test = np.load("X_TEST.npy")
y_test = np.load("Y_TEST.npy")


In [5]:
model = load_model("new_model.h5")

In [6]:
#To test accuracy of trained model on test data
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)
print("Accuracy:" + str(acc))

145/145 [==============================] - 3s 22ms/step
Accuracy:0.9793103337287903


In [7]:
x_test.shape

(145, 400, 400, 1)

In [8]:
y_test.shape

(145, 6)

In [9]:
print(reverselookup[np.argmax(model.predict(x_test[1].reshape(1,400,400,1)))])

Palm


In [10]:
print(reverselookup[np.argmax(y_test[1])])

Palm


In [13]:
"""cap  = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    #To flip the image if needed
    #frame = cv2.flip(frame,1)
    
    cv2.rectangle(frame,(0,100),(300,400),(0,255,0))
    roi = frame[100:400,0:300]
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    kernel = np.ones((3,3))
    
    #Skin color range in hsv
    lower_skin = np.array([0,20,70])
    upper_skin = np.array([20,255,255])
    
    #Capture hand in the region of interest and convert hand pixels to white and rest to black pixels
    mask = cv2.inRange(hsv,lower_skin,upper_skin)
    
    #extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations=4)
    
    #Blurring the image
    mask = cv2.GaussianBlur(mask,(5,5),100)
    #mask = cv2.bitwise_and(roi,roi,mask=mask) 
    #mask_gray = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    
    mask_resized = cv2.resize(mask,(400,400))
    mask_reshaped = mask_resized.reshape(1,400,400,1)/255.0
    cv2.putText(frame, reverselookup[np.argmax(model.predict(mask_reshaped))], (0,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    
    
    
    
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()"""

In [10]:
# Data creation

cap = cv2.VideoCapture(0)
flag = True
count = 1
print("Select option" + str(reverselookup))
option = int(input())
_, first_frame = cap.read()
roi = first_frame[100:400,0:300]
first_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
first_roi = cv2.GaussianBlur(first_roi, (5, 5), 0)



while True:
    _,frame = cap.read()
    
    if flag:
        if cv2.waitKey(60) & 0xFF == ord('r'):
            flag = False
        else:
            cv2.rectangle(frame,(0,100),(300,400),(0,255,0))
            roi = frame[100:400,0:300]
    
            gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
    
            difference = cv2.absdiff(first_roi, gray_roi)
            #_, difference = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)
            cv2.imshow("difference",difference)
            
            
    else:
        cv2.rectangle(frame,(0,100),(300,400),(0,255,0))
        roi = frame[100:400,0:300]
    
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
    
        difference = cv2.absdiff(first_roi, gray_roi)
        #_, difference = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)
        cv2.imshow("difference",difference)
        cv2.imwrite('dataset/'+reverselookup[option] +'/image' + str(count) + '.png', difference)
        count += 1
        if count%100 == 0:
            print(count)     
    
    if cv2.waitKey(200) & 0xFF == 27: # you can increase delay to 2 seconds here
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Select option{0: 'Fist', 1: 'Nothing', 2: 'Palm', 3: 'Swing', 4: 'Thumb', 5: 'Yo'}
2


In [12]:
#Running model
cap  = cv2.VideoCapture(0)

_, first_frame = cap.read()
cv2.rectangle(first_frame,(0,100),(300,400),(0,255,0))
roi = first_frame[100:400,0:300]
first_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
first_roi = cv2.GaussianBlur(first_roi, (5, 5), 0)

while True:
    _,frame = cap.read()
    #To flip the image if needed
    #frame = cv2.flip(frame,1)
    
    cv2.rectangle(frame,(0,100),(300,400),(0,255,0))
    roi = frame[100:400,0:300]
    
    
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
    
    difference = cv2.absdiff(first_roi, gray_roi)
    _, difference = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)
    
    difference_resized = cv2.resize(difference,(400,400))
    difference_reshaped = difference_resized.reshape(1,400,400,1)/255.0
    cv2.putText(frame, reverselookup[np.argmax(model.predict(difference_reshaped))], (0,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    
    
    
    
    
    cv2.imshow("first_frame",first_frame)
    cv2.imshow("frame",frame)
    cv2.imshow("difference",difference)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()